Este projeto tem como finalidade apresentar gráficos para uma gerência de uma empresa fictícia chamada Empresa Gráfica Visconde (EGV), sobre o desempenho do setor de informática da empresa.
O setor de informáfica tem como responsabilidade uma empresa tercerizada chamada SoftBahia.
Estes gráficos são necessários para uma análise mensal, os dados produzidos pelo sistema GLPI, sistema usado pelo setor para o atendimento de suas funções. Dessa forma, a gerência de informática tem como objetivo produzir estes gráficos para a transmissão do conhecimento da produtividade mensal.

In [211]:
import pandas as pd
import plotly.express as px

In [212]:
dados = pd.read_csv('RelatorioDezembro.csv', sep=',')
dados.head(1)

,ID,Título,Tipo,Status,Data de abertura,Requerente - Requerente,Requerente - Grupo requerente,Atribuído - Técnico,Atribuído - Grupo técnico,Categoria,Localização,Requerente - Autor,Última atualização,Tempo para atendimento + Progresso,Tempo para solução + Progresso,Time to resolve exceeded,Estatísticas - Tempo de solução,SLAs - SLAs Tempo para solução,Tempo de solução - Horas:Minutos
0,2 201 044 171,Falha na restauração do backup de banco de dad...,Incidente,Em atendimento (atribuído),2023-12-19 08:05:00,Rael Santista (299),GERINF > SSR,Israel Espadao (716),Gerencia de Informatica > Suporte N3,Gerenciamento de Banco de Dados > BANCO DE DADOS,Gerinf - SSR,Jade Roque (722),2023-12-26 13:13:00,2023-12-19 08:25:00,2023-12-12 13:09:00,Sim,0 segundo,Urgente,00:00


# 1. Tratamento de Dados

### 1.1 Corrigindo dados mal tratados

In [213]:
dados['Tempo para solução + Progresso'] = pd.to_datetime(dados['Tempo para solução + Progresso'], format='%d/%m/%Y %H:%M', errors='coerce')

In [214]:
mediana = dados['Tempo para solução + Progresso'].median()

c:\Users\paulo\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\lib\nanfunctions.py:1215: RuntimeWarning:

Mean of empty slice



In [215]:
# Substitui os valores NaN pela mediana
dados['Tempo para solução + Progresso'].fillna(mediana, inplace=True)

In [216]:
dados['Tempo para solução + Progresso']

0     NaT
1     NaT
2     NaT
3     NaT
4     NaT
       ..
553   NaT
554   NaT
555   NaT
556   NaT
557   NaT
Name: Tempo para solução + Progresso, Length: 558, dtype: datetime64[ns]

### 1.2 Transformando alguns dados em formato dia/mês/ano hora:minuto

In [217]:
dados['Data de abertura'] = pd.to_datetime(dados['Data de abertura'], format= '%Y-%m-%d %H:%M:%S')
dados['Última atualização'] = pd.to_datetime(dados['Última atualização'], format='%Y-%m-%d %H:%M:%S')
dados['Tempo para atendimento + Progresso'] = pd.to_datetime(dados['Tempo para atendimento + Progresso'], format='%Y-%m-%d %H:%M:%S')

### 1.3 Transformando dos dados da coluna "Estatísticas - Tempo de solução" em formato hora:minuto

In [218]:
# Criando uma função para os dados em tempo numérico.
def converte_tempo(tempo_str):
  dias, horas, minutos = 0, 0, 0

  if 'dia' in tempo_str:
    dias = int(tempo_str.split('dia')[0].strip())
  if 'hora' in tempo_str:
    horas = int(tempo_str.split('hora')[0].split()[-1].strip())
  if 'minuto' in tempo_str:
    minutos = int(tempo_str.split('minuto')[0].split()[-1].strip())

  # Converte os dias em horas e faz o somatório.
  total_horas = dias * 24 + horas

  # Formata o resultado para Horas:Minutos
  resultado_formatado = f"{total_horas:02d}:{minutos:02d}"
  return resultado_formatado

### 1.4 Aplica o tempo formatado em Horas:Minutos

In [219]:
dados['Tempo de solução - Horas:Minutos'] = dados['Estatísticas - Tempo de solução'].apply(converte_tempo)
dados.head(1)

,ID,Título,Tipo,Status,Data de abertura,Requerente - Requerente,Requerente - Grupo requerente,Atribuído - Técnico,Atribuído - Grupo técnico,Categoria,Localização,Requerente - Autor,Última atualização,Tempo para atendimento + Progresso,Tempo para solução + Progresso,Time to resolve exceeded,Estatísticas - Tempo de solução,SLAs - SLAs Tempo para solução,Tempo de solução - Horas:Minutos
0,2 201 044 171,Falha na restauração do backup de banco de dad...,Incidente,Em atendimento (atribuído),2023-12-19 08:05:00,Rael Santista (299),GERINF > SSR,Israel Espadao (716),Gerencia de Informatica > Suporte N3,Gerenciamento de Banco de Dados > BANCO DE DADOS,Gerinf - SSR,Jade Roque (722),2023-12-26 13:13:00,2023-12-19 08:25:00,NaT,Sim,0 segundo,Urgente,00:00


### 1.5 Gerando tabela tratada

In [220]:
path_file_geral = 'TabelaNova.csv'
dados.to_csv(path_file_geral, index=False)

In [221]:
# Remover quebras de linha na coluna específica
dados['Requerente - Grupo requerente'] = dados['Requerente - Grupo requerente'].str.replace('\n', ' ')
dados['Requerente - Grupo requerente'][453]

'GERAG'

### 1.6 Temporalidade das Categorias. Categoria x Somatória do tempo

In [222]:
# Convertendo a coluna 'TempoGasto' para o tipo timedelta usando apply
dados['Tempo de solução - Horas:Minutos'] = dados['Tempo de solução - Horas:Minutos'].apply(lambda x: pd.to_timedelta(x + ':00'))

# Calculando a soma dos tempos gastos para cada categoria em segundos
soma_segundos_por_categoria = dados.groupby('Categoria')['Tempo de solução - Horas:Minutos'].sum().dt.total_seconds().reset_index()

# Convertendo a soma de segundos de volta para o formato HH:MM
soma_segundos_por_categoria['Tempo de solução - Horas:Minutos'] = pd.to_datetime(soma_segundos_por_categoria['Tempo de solução - Horas:Minutos'], unit='s').dt.time
soma_segundos_por_categoria

,Categoria,Tempo de solução - Horas:Minutos
0,Ambientes Computacionais > ACTIVE DIRECTORY,14:52:00
1,Ambientes Computacionais > APLICATIVOS DIVERSOS,09:48:00
2,Ambientes Computacionais > APLICATIVOS DIVERSO...,06:26:00
3,Ambientes Computacionais > APLICATIVOS DIVERSO...,02:32:00
4,Ambientes Computacionais > APLICATIVOS DIVERSO...,02:15:00
5,Ambientes Computacionais > APOIO TÉCNICO,22:41:00
6,Ambientes Computacionais > BACKUP,04:31:00
7,Ambientes Computacionais > DHCP,01:37:00
8,Ambientes Computacionais > ESTAÇÕES DE TRABALHO,05:20:00
9,Ambientes Computacionais > IMPRESSÃO CORPORATIVA,15:45:00


In [223]:
# Caminho do novo arquivo Excel (xlsx)
path_file = 'NovoRelatorio_SomatorioPorSetor.xlsx'
soma_segundos_por_categoria.to_excel(path_file, index=False)

# 2. Nível de Serviço Contratado X Nível de Serviço Realizado

### 2.0.1. Agrupando dados do serviço realizado.

In [224]:
servicos_realizado = dados.groupby(['SLAs - SLAs Tempo para solução', 'Time to resolve exceeded']).size().reset_index(name='Contagem')
servicos_realizado

,SLAs - SLAs Tempo para solução,Time to resolve exceeded,Contagem
0,Crítico,Não,1
1,Crítico,Sim,1
2,Imediatos,Não,2
3,Imediatos,Sim,1
4,Normal,Não,455
5,Normal,Sim,7
6,Programados,Não,12
7,Urgente,Não,75
8,Urgente,Sim,4


### 2.0.2. Serviços realizados na sua totalidade

In [225]:
servicos_total = servicos_realizado.groupby('SLAs - SLAs Tempo para solução')['Contagem'].sum().reset_index(name='Servicos_totais')
servicos_total

,SLAs - SLAs Tempo para solução,Servicos_totais
0,Crítico,2
1,Imediatos,3
2,Normal,462
3,Programados,12
4,Urgente,79


### 2.0.3. Serviços realizados no prazo

In [226]:
servicos_noprazo = servicos_realizado[servicos_realizado['Time to resolve exceeded'] == 'Não']
servicos_noprazo = servicos_noprazo.groupby('SLAs - SLAs Tempo para solução')['Contagem'].sum().reset_index(name='Servicos_noprazo')
servicos_noprazo

,SLAs - SLAs Tempo para solução,Servicos_noprazo
0,Crítico,1
1,Imediatos,2
2,Normal,455
3,Programados,12
4,Urgente,75


### 2.0.4. Serviço no prazo em percentual

In [227]:
servicos_percentagem = (servicos_noprazo['Servicos_noprazo'] * 100)/ (servicos_total['Servicos_totais'])


In [228]:
percentagem_df = pd.DataFrame({
    'SLAs - SLAs Tempo para solução': ['Crítico', 'Imediatos', 'Normal', 'Programados', 'Urgente'],
    'servicos_percentagem': servicos_percentagem.round(2)
})
percentagem_df

,SLAs - SLAs Tempo para solução,servicos_percentagem
0,Crítico,50.00
1,Imediatos,66.67
2,Normal,98.48
3,Programados,100.00
4,Urgente,94.94


### 2.0.5. Serviço Contratado

In [229]:
# Serviço Contratado
servicos_contratado = [98.5, 99, 95, 100, 98]
# Se servico_contratado é uma lista
contratados_df = pd.DataFrame({
    'SLAs - SLAs Tempo para solução': ['Crítico', 'Imediatos', 'Normal', 'Programados', 'Urgente'],
    'servico_contratado': servicos_contratado
})
contratados_df

,SLAs - SLAs Tempo para solução,servico_contratado
0,Crítico,98.5
1,Imediatos,99.0
2,Normal,95.0
3,Programados,100.0
4,Urgente,98.0


### 2.0.6. Combinações de tabelas

In [230]:
combined_df = pd.merge(servicos_total, servicos_noprazo, on='SLAs - SLAs Tempo para solução', how='outer')
combined_df

,SLAs - SLAs Tempo para solução,Servicos_totais,Servicos_noprazo
0,Crítico,2,1
1,Imediatos,3,2
2,Normal,462,455
3,Programados,12,12
4,Urgente,79,75


In [231]:
combined_df1 = pd.merge(contratados_df, percentagem_df, on='SLAs - SLAs Tempo para solução', how='outer')
combined_df1

,SLAs - SLAs Tempo para solução,servico_contratado,servicos_percentagem
0,Crítico,98.5,50.00
1,Imediatos,99.0,66.67
2,Normal,95.0,98.48
3,Programados,100.0,100.00
4,Urgente,98.0,94.94


In [232]:
combined_all_dfs = pd.merge(combined_df, combined_df1, on='SLAs - SLAs Tempo para solução', how='outer')
combined_all_dfs

,SLAs - SLAs Tempo para solução,Servicos_totais,Servicos_noprazo,servico_contratado,servicos_percentagem
0,Crítico,2,1,98.5,50.00
1,Imediatos,3,2,99.0,66.67
2,Normal,462,455,95.0,98.48
3,Programados,12,12,100.0,100.00
4,Urgente,79,75,98.0,94.94


## 2.1. Gráfico 1: Nível de Serviço Contratado X Nível de Serviço Realizado

In [233]:
graf1 = px.bar(
    combined_all_dfs,
    x='SLAs - SLAs Tempo para solução',
    y=['servico_contratado', 'servicos_percentagem'],
    barmode='group',
    labels={'value': 'Contagem'},
    text='value',
    title='Nível de Serviço Contratado X Nível de Serviço Realizado'
)

graf1.update_traces(textposition='outside')

graf1.show()

# 3. RESULTADOS APURADOS DOS SERVIÇOS

## 3.1. Percentual por tipo de chamado

In [234]:
# agrupando dados
tipos_chamados = dados.groupby('Tipo').size().reset_index(name='quantidade')
# criando gráfico
graf2 = px.pie(tipos_chamados, names='Tipo', values='quantidade', title='Percentual por tipo de chamado')
graf2.update_traces(textinfo='value', textposition='outside')
graf2.show()

## 3.2. Percentual de Incidentes e Solicitações de Serviços registradas por prioridade relatada.


### 3.2.1. Incidentes por Prioridade

In [235]:
# Agrupando os dados
incidentes = dados.groupby(['SLAs - SLAs Tempo para solução', 'Tipo']).size().reset_index(name='ContagemIncidente')

# Filtrando apenas os dados de incidente
incidente_data = incidentes[incidentes['Tipo'] == 'Incidente']

# Criando o gráfico
graf3 = px.pie(incidente_data, names='SLAs - SLAs Tempo para solução', values='ContagemIncidente', title='Incidentes por Prioridade')

graf3.update_traces(textinfo='value', textposition='outside')

graf3.show()

### 3.2.2. Solicitações por Prioridade

In [236]:
# Agrupando os dados
requisicoes = dados.groupby(['SLAs - SLAs Tempo para solução', 'Tipo']).size().reset_index(name='Contagem')

# Filtrando apenas os dados de requisição
requisicao_data = requisicoes[requisicoes['Tipo'] == 'Requisição']

# Criando o gráfico
graf4 = px.pie(requisicao_data, names='SLAs - SLAs Tempo para solução', values='Contagem', title='Solicitações por Prioridade', hover_data=['Contagem'])

graf4.update_traces(textinfo='value', textposition='outside')

graf4.show()

## 3.3. Atendimentos por área demandante

### 3.3.1 Solicitações abertas por Setor

In [237]:
requisicoes = dados[(dados['Localização'].notnull()) & (dados['Tipo'] == 'Requisição')]

contagem_por_localizacao = requisicoes.groupby('Localização').size().reset_index(name='Quantidade')

contagem_por_localizacao = contagem_por_localizacao.sort_values(by='Quantidade', ascending=False)

In [238]:
graf5 = px.bar(contagem_por_localizacao, x=contagem_por_localizacao['Localização'], y=contagem_por_localizacao['Quantidade'],
               title='Solicitações abertas por áreas demandantes',
               text=contagem_por_localizacao['Quantidade'])



graf5.update_traces(texttemplate='%{text}', textposition='outside')

graf5.show()

### 3.3.2. Incidentes abertos por Setor

In [239]:
incidentes = dados[(dados['Localização'].notnull()) & (dados['Tipo'] == 'Incidente')]

contagem_por_localizacao_inc = incidentes.groupby('Localização').size().reset_index(name='Quantidade')

contagem_por_localizacao_inc = contagem_por_localizacao_inc.sort_values(by='Quantidade', ascending=False)

In [240]:
graf6 = px.bar(x=contagem_por_localizacao_inc['Localização'], y=contagem_por_localizacao_inc['Quantidade'],
               title='Incidentes abertos por área demandante',
               text=contagem_por_localizacao_inc['Quantidade'])

graf6.update_traces(texttemplate='%{text}', textposition='outside')

graf6.show()

## 3.4. Incidentes por níveis de atendimento

In [241]:
dados_atendimento = dados.groupby(['Atribuído - Grupo técnico', 'Status', 'Tipo']).size().reset_index(name='Contagem')
dados_atendimento

,Atribuído - Grupo técnico,Status,Tipo,Contagem
0,GERINF > SAD,Em atendimento (atribuído),Requisição,1
1,GERINF > SAD,Fechado,Incidente,18
2,GERINF > SAD,Fechado,Requisição,31
3,GERINF > SSR,Em atendimento (planejado),Requisição,3
4,GERINF > SSR,Fechado,Incidente,34
5,GERINF > SSR,Fechado,Requisição,30
6,Gerencia de Informatica > Fornecedor,Em atendimento (atribuído),Incidente,1
7,Gerencia de Informatica > Fornecedor,Fechado,Incidente,3
8,Gerencia de Informatica > Fornecedor,Fechado,Requisição,1
9,Gerencia de Informatica > Fornecedor,Pendente,Incidente,1


In [242]:
incidentes_fechados = dados_atendimento[(dados_atendimento['Tipo'] == 'Incidente') & (dados_atendimento['Status'] == 'Fechado')]
incidentes_total = dados_atendimento[dados_atendimento['Tipo'] == 'Incidente']

In [243]:
incidentes_total = incidentes_total.groupby('Atribuído - Grupo técnico')['Contagem'].sum().reset_index(name='Incidentes_total')
incidentes_fechados = incidentes_fechados.groupby('Atribuído - Grupo técnico')['Contagem'].sum().reset_index(name='Incidentes_fechados')

In [244]:
combined_df1 = pd.merge(incidentes_fechados, incidentes_total, on='Atribuído - Grupo técnico', how='outer', suffixes=('_fechados', '_total'))
combined_df1

,Atribuído - Grupo técnico,Incidentes_fechados,Incidentes_total
0,GERINF > SAD,18,18
1,GERINF > SSR,34,34
2,Gerencia de Informatica > Fornecedor,3,5
3,Gerencia de Informatica > Suporte N1,11,11
4,Gerencia de Informatica > Suporte N2,79,79
5,Gerencia de Informatica > Suporte N3,105,106


In [245]:
combined_df1['Incidentes_fechados'] = combined_df1['Incidentes_fechados'].fillna(0).astype(int)

### 3.4.1. Gráfico de **incidentes** por nível de atendimento

In [246]:
# Criar o gráfico de barras combinado
graf7 = px.bar(
    combined_df1,
    x='Atribuído - Grupo técnico',
    y=['Incidentes_total', 'Incidentes_fechados'],
    barmode='group',
    labels={'value': 'Contagem'},
    text='value',
    title='Incidentes por grupo técnico'
)

graf7.update_traces(textposition='outside')


# Exibir o gráfico
graf7.show()

## 3.5. Solicitações por nível de atendimento

In [247]:
solicitacoes_fechados = dados_atendimento[(dados_atendimento['Status'] == 'Fechado') & (dados_atendimento['Tipo'] == 'Requisição')]
solicitacoes_total = dados_atendimento[(dados_atendimento['Tipo'] == 'Requisição')]

In [248]:
solicitacoes_fechados = solicitacoes_fechados.groupby('Atribuído - Grupo técnico')['Contagem'].sum().reset_index(name='Contagem_fechados')
solicitacoes_total = solicitacoes_total.groupby('Atribuído - Grupo técnico')['Contagem'].sum().reset_index(name='Contagem_total')

In [249]:
combined_df = pd.merge(solicitacoes_fechados, solicitacoes_total, on='Atribuído - Grupo técnico', how='outer')
combined_df

,Atribuído - Grupo técnico,Contagem_fechados,Contagem_total
0,GERINF > SAD,31,32
1,GERINF > SSR,30,33
2,Gerencia de Informatica > Fornecedor,1,1
3,Gerencia de Informatica > Suporte N2,37,37
4,Gerencia de Informatica > Suporte N3,200,202


In [250]:
combined_df['Atribuído - Grupo técnico'] = combined_df['Atribuído - Grupo técnico'].astype('category')

### 3.5.1. Gráfico de **solicitações** por nível de atendimento

In [251]:
graf11 = px.bar(
    combined_df,
    x='Atribuído - Grupo técnico',
    y=['Contagem_total', 'Contagem_fechados'],
    barmode='group',
    labels={'value': 'Contagem'},
    text='value',
    title='Solicitações por grupo técnico'
)

graf11.update_traces(textposition='outside')

# Exibir o gráfico
graf11.show()

# 4. Análise por técnicos

## 4.1. Filtrando por **requisição**

In [252]:
tipo_requisicao = dados.loc[dados['Tipo'] == 'Requisição', 'Tipo']

In [253]:

requisicao_por_tecnico = dados.groupby(['Requerente - Autor', 'SLAs - SLAs Tempo para solução', tipo_requisicao, 'Time to resolve exceeded']).size().reset_index(name='Contagem')
requisicao_por_tecnico

,Requerente - Autor,SLAs - SLAs Tempo para solução,Tipo,Time to resolve exceeded,Contagem
0,Inacius Emmanuel Fatal (735),Normal,Requisição,Não,90
1,Inacius Emmanuel Fatal (735),Programados,Requisição,Não,5
2,Inacius Emmanuel Fatal (735),Urgente,Requisição,Não,18
3,Inacius Emmanuel Fatal (735),Urgente,Requisição,Sim,1
4,Jade Roque (722),Normal,Requisição,Não,174
5,Jade Roque (722),Normal,Requisição,Sim,1
6,Jade Roque (722),Programados,Requisição,Não,7
7,Jade Roque (722),Urgente,Requisição,Não,9


### 4.1.1. Filtrando por demanda atendida no prazo

In [254]:
tempo_excedido = dados.loc[dados['Time to resolve exceeded'] == 'Não', 'Time to resolve exceeded']

In [255]:
requisicao_por_tecnico_no_prazo = dados.groupby(['Requerente - Autor', 'SLAs - SLAs Tempo para solução', tipo_requisicao, tempo_excedido]).size().reset_index(name='Contagem')
requisicao_por_tecnico_no_prazo

,Requerente - Autor,SLAs - SLAs Tempo para solução,Tipo,Time to resolve exceeded,Contagem
0,Inacius Emmanuel Fatal (735),Normal,Requisição,Não,90
1,Inacius Emmanuel Fatal (735),Programados,Requisição,Não,5
2,Inacius Emmanuel Fatal (735),Urgente,Requisição,Não,18
3,Jade Roque (722),Normal,Requisição,Não,174
4,Jade Roque (722),Programados,Requisição,Não,7
5,Jade Roque (722),Urgente,Requisição,Não,9


## 4.2. Filtrando por **incidente**

In [256]:
tipo_incidente = dados.loc[dados['Tipo'] == 'Incidente', 'Tipo']

In [257]:
incidente_por_tecnico = dados.groupby(['Requerente - Autor', 'SLAs - SLAs Tempo para solução', tipo_incidente, 'Time to resolve exceeded']).size().reset_index(name='Contagem')
incidente_por_tecnico


,Requerente - Autor,SLAs - SLAs Tempo para solução,Tipo,Time to resolve exceeded,Contagem
0,Boris Ianovich (723),Normal,Incidente,Não,1
1,Inacius Emmanuel Fatal (735),Crítico,Incidente,Não,1
2,Inacius Emmanuel Fatal (735),Normal,Incidente,Não,102
3,Inacius Emmanuel Fatal (735),Normal,Incidente,Sim,2
4,Inacius Emmanuel Fatal (735),Urgente,Incidente,Não,21
5,Jade Roque (722),Crítico,Incidente,Sim,1
6,Jade Roque (722),Imediatos,Incidente,Não,2
7,Jade Roque (722),Imediatos,Incidente,Sim,1
8,Jade Roque (722),Normal,Incidente,Não,88
9,Jade Roque (722),Normal,Incidente,Sim,4


### 4.2.1. Filtrando por demanda atendida no prazo

In [258]:
tempo_excedido_incidente = dados.groupby(['Requerente - Autor', 'SLAs - SLAs Tempo para solução', tipo_incidente, tempo_excedido]).size().reset_index(name='Contagem')
tempo_excedido_incidente

,Requerente - Autor,SLAs - SLAs Tempo para solução,Tipo,Time to resolve exceeded,Contagem
0,Boris Ianovich (723),Normal,Incidente,Não,1
1,Inacius Emmanuel Fatal (735),Crítico,Incidente,Não,1
2,Inacius Emmanuel Fatal (735),Normal,Incidente,Não,102
3,Inacius Emmanuel Fatal (735),Urgente,Incidente,Não,21
4,Jade Roque (722),Imediatos,Incidente,Não,2
5,Jade Roque (722),Normal,Incidente,Não,88
6,Jade Roque (722),Urgente,Incidente,Não,27


# 5. Quantidade de chamado por categoria. Incidentes e Solicitação.

## 5.1. Incidentes

In [259]:
chamado_incidente = dados.groupby([tipo_incidente, 'Categoria']).size().reset_index(name='Contagem')
chamado_incidente

,Tipo,Categoria,Contagem
0,Incidente,Ambientes Computacionais > ACTIVE DIRECTORY,33
1,Incidente,Ambientes Computacionais > APLICATIVOS DIVERSOS,9
2,Incidente,Ambientes Computacionais > APLICATIVOS DIVERSO...,2
3,Incidente,Ambientes Computacionais > APLICATIVOS DIVERSO...,5
4,Incidente,Ambientes Computacionais > APLICATIVOS DIVERSO...,2
5,Incidente,Ambientes Computacionais > APOIO TÉCNICO,16
6,Incidente,Ambientes Computacionais > BACKUP,3
7,Incidente,Ambientes Computacionais > DHCP,10
8,Incidente,Ambientes Computacionais > ESTAÇÕES DE TRABALHO,23
9,Incidente,Ambientes Computacionais > IMPRESSÃO CORPORATIVA,34


## 5.2. Requisições

In [260]:
chamado_requisicao = dados.groupby([tipo_requisicao, 'Categoria']).size().reset_index(name='Contagem')
chamado_requisicao

,Tipo,Categoria,Contagem
0,Requisição,Ambientes Computacionais > ACTIVE DIRECTORY,33
1,Requisição,Ambientes Computacionais > APLICATIVOS DIVERSOS,1
2,Requisição,Ambientes Computacionais > APLICATIVOS DIVERSO...,1
3,Requisição,Ambientes Computacionais > APOIO TÉCNICO,1
4,Requisição,Ambientes Computacionais > BACKUP,1
5,Requisição,Ambientes Computacionais > ESTAÇÕES DE TRABALHO,37
6,Requisição,Ambientes Computacionais > S.O. - SERVIDORES,4
7,Requisição,Ambientes Computacionais > SERVIÇOS WEB,1
8,Requisição,Ambientes Computacionais > SERVIÇOS WEB > EGBANET,1
9,Requisição,Ambientes Computacionais > SERVIÇOS WEB > INTR...,2


# 6. Designados X Finalizados, por **tipo**

## 6.1. Incidentes

### 6.1.1 Tratamento de dados

In [261]:
tratamento_status = {'Em atendimento (atribuído)': 'Pendente', 'Em atendimento (atribuído)': 'Pendente'}

# Substitui os valores na coluna 'Estados' usando o mapeamento
dados['Status'] = dados['Status'].replace(tratamento_status)

In [262]:
df_dxf_total = dados.groupby(['Tipo', 'Status']).size().reset_index(name='Contagem')
df_dxf_total

,Tipo,Status,Contagem
0,Incidente,Fechado,250
1,Incidente,Pendente,3
2,Requisição,Em atendimento (planejado),3
3,Requisição,Fechado,299
4,Requisição,Pendente,3


In [263]:
df_dxf_tipo = dados.groupby(['Tipo']).size().reset_index(name='Contagem')
df_dxf_tipo

,Tipo,Contagem
0,Incidente,253
1,Requisição,305


In [264]:
drop_df_dxf_tipo = df_dxf_tipo.drop(1)
drop_df_dxf_tipo

,Tipo,Contagem
0,Incidente,253


In [265]:
dxf_incidentes_fechados = df_dxf_total[(df_dxf_total['Status'] == 'Fechado') & (df_dxf_total['Tipo'] == 'Incidente')]
dxf_incidentes_fechados

,Tipo,Status,Contagem
0,Incidente,Fechado,250


In [266]:
dxf_incidentes_totais = df_dxf_total[(df_dxf_total['Tipo'] == 'Incidente')]
dxf_incidentes_totais

,Tipo,Status,Contagem
0,Incidente,Fechado,250
1,Incidente,Pendente,3


In [267]:
drop_status = dxf_incidentes_totais.drop('Status', axis=1)
drop_status

,Tipo,Contagem
0,Incidente,250
1,Incidente,3


In [268]:
drop_linha1 = drop_status.drop(1)
drop_linha1

,Tipo,Contagem
0,Incidente,250


In [269]:
merge_dxf = pd.merge(drop_df_dxf_tipo, drop_linha1, on=['Tipo', 'Contagem'], how='outer')
merge_dxf

,Tipo,Contagem
0,Incidente,253
1,Incidente,250


In [270]:
edit_linha_tipo = merge_dxf.at[0, 'Tipo'] = 'Total de Incidentes'
merge_dxf

,Tipo,Contagem
0,Total de Incidentes,253
1,Incidente,250


## 6.1.2 Gráfico de abertos por **incidente**

In [271]:
abertos_por_incidente = px.pie(merge_dxf, names='Tipo', values='Contagem', title='Abertos por Incidente')
abertos_por_incidente.update_traces(textinfo='value', textposition='outside')
abertos_por_incidente.show()

## 6.2. Requisições

### 6.2.1. Tratamento de dados



In [272]:
df_dxf_tipo = dados.groupby(['Tipo']).size().reset_index(name='Contagem')
df_dxf_tipo

,Tipo,Contagem
0,Incidente,253
1,Requisição,305


In [273]:
drop_df_dxf_tipo_req = df_dxf_tipo.drop(0)
drop_df_dxf_tipo_req

,Tipo,Contagem
1,Requisição,305


In [274]:
dxf_requisicao_fechados = df_dxf_total[(df_dxf_total['Status'] == 'Fechado') & (df_dxf_total['Tipo'] == 'Requisição')]
dxf_requisicao_fechados

,Tipo,Status,Contagem
3,Requisição,Fechado,299


In [275]:
dxf_requisicao_totais = df_dxf_total[(df_dxf_total['Tipo'] == 'Requisição')]
dxf_requisicao_totais

,Tipo,Status,Contagem
2,Requisição,Em atendimento (planejado),3
3,Requisição,Fechado,299
4,Requisição,Pendente,3


In [276]:
drop_status = dxf_requisicao_totais.drop('Status', axis=1)
drop_status

,Tipo,Contagem
2,Requisição,3
3,Requisição,299
4,Requisição,3


In [277]:
drop_status1 = dxf_requisicao_fechados.drop('Status', axis=1)
drop_status1

,Tipo,Contagem
3,Requisição,299


In [278]:
merge_dxf1 = pd.merge(drop_df_dxf_tipo_req, drop_status1, on=['Tipo', 'Contagem'], how='outer')
merge_dxf1

,Tipo,Contagem
0,Requisição,305
1,Requisição,299


In [279]:
edit_linha_tipo_req = merge_dxf1.at[0, 'Tipo'] = 'Total de Requisições'
merge_dxf1

,Tipo,Contagem
0,Total de Requisições,305
1,Requisição,299


## 6.2.2. Gráfico de abertos por **requisição**

In [280]:
abertos_por_requisicao = px.pie(merge_dxf1, names='Tipo', values='Contagem', title='Abertos por requisição')
abertos_por_requisicao.update_traces(textinfo='value', textposition='outside')
abertos_por_requisicao.show()